In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import inspect
import os
import sys
import time

np.random.seed(42) # nothing special about this seed (we hope)!

In [5]:
import bnpregcluster_runjingdev.regression_mixture_lib as gmm_lib

ImportError: No module named 'aistats2019_ij_paper'